In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/00.config/config

In [0]:
%run /Workspace/Users/valterlafuentejunior@gmail.com/ProjetoMedalhao/Utils/utils


In [0]:
from pyspark.sql.functions import to_date, col, date_format, regexp_replace
from delta.tables import DeltaTable

In [0]:
tabela_destino = "workspace.silver_etl.economia"


In [0]:
# Leitura das bases Bronzes
ipca = spark.table("workspace.bronze_etl.ipca")
boi_gordo = spark.table("workspace.bronze_etl.boi_gordo")


In [0]:
boi_gordo = boi_gordo.withColumnRenamed("Data", "data").withColumnRenamed("Valor", "boi_gordo")


In [0]:
boi_gordo = boi_gordo.withColumn("data", to_date("data", "MM/yyyy"))


In [0]:
boi_gordo = boi_gordo.withColumn("data", date_format("data", "yyyy-MM"))


In [0]:
ipca = ipca.withColumn("data", to_date("data", "dd/MM/yyyy"))


In [0]:
ipca = ipca.withColumn("data", date_format("data", "yyyy-MM"))


In [0]:
ip = ipca.alias("ip")
bo = boi_gordo.alias("bo")

In [0]:
df_join = ip.join(bo, col("ip.data") == col("bo.data"), "inner") \
              .select(
                  col("ip.data").alias("data"),
                  col("ip.ipca"),
                  col("bo.boi_gordo").alias("boi_gordo"),
                  col("ip.data_coleta").alias("data_coleta"),
              )

In [0]:
df_join = df_join.dropDuplicates(["data_coleta", "data"])

In [0]:
df_join = df_join.withColumn("data", to_date(col("data"), "yyyy-MM"))


In [0]:
df_final = df_join.withColumn("boi_gordo", regexp_replace(col("boi_gordo"), ",", ".").cast("double"))

In [0]:
merge_delta_table(df_join, tabela_destino)